In [1]:
# Import appropriate libraries and packages
from pathlib import Path
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
import torch.optim as optim
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *
from cmd_classes_funcs_Marchese import *

/home/CAMPUS/cemb2020/anaconda3/envs/arcslab/lib/python3.8/site-packages/fastbook/__init__.py:18: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [2]:
# Set GPU to run on if available
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=3)

In [3]:
# Get classes and filenames
path = Path("data")
classes = get_class_labels(path)
all_filenames = get_filenames(path)

## Splitting Data into train/validate sets

In [4]:
# Getting size of dataset and corresponding list of indices
dataset_size = len(all_filenames)
dataset_indices = list(range(dataset_size))

In [5]:
# Shuffling the indices
np.random.shuffle(dataset_indices)

In [6]:
# Getting index for where we want to split the data
val_split_index = int(np.floor(0.2 * dataset_size))

In [7]:
# Splitting list of indices into training and validation indices
train_idx, val_idx = dataset_indices[val_split_index:], dataset_indices[:val_split_index]

In [8]:
# Creating samplers
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [9]:
# Getting list of filenames for training and validation set
train_filenames = [all_filenames[i] for i in train_idx]
val_filenames = [all_filenames[i] for i in val_idx]

In [10]:
# Create training and validation datasets
train_data = ImageWithCmdDataset(classes, train_filenames)
val_data = ImageWithCmdDataset(classes, val_filenames)

In [11]:
# Create the training set and validation set data loaders
train_loader = DataLoader(dataset=train_data, shuffle=False, batch_size=16, sampler=train_sampler)
val_loader = DataLoader(dataset=val_data, shuffle=False, batch_size=16, sampler=val_sampler)

In [12]:
# Instantiate the network
net = MyModel_next50()

In [13]:
# Send model to GPU
net.to(device)

MyModel_next50(
  (cnn): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [14]:
####n = dataset_size
#### w = torch.tensor([(n-138)/n,(n-211)/n,(n-786)/n])
#### w = w.to(device)
# defining loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=w)
optimizer = optim.Adam(net.parameters(), lr=.0001)

In [15]:
num_epochs = 50

In [16]:
# Import time package to keep track of training time
from time import time

In [17]:
# Model training

net.train()

for epoch in range(num_epochs):  

    running_loss = 0.0
    
    start = time()
    
    for data in train_loader:
        # Get the inputs and labels
        inp_data, label = data
        
        # Break up the inputs
        img, cmd = inp_data
        
        # Putting data into the GPU
        img = img.to(device)
        cmd = cmd.to(device)
        label = label.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward + backward + optimize
        output = net((img, cmd))
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        # Add loss of current data to running loss
        running_loss += loss.item()
        
    # Print statistics    
    print(f"Epoch:{epoch+1:}/{num_epochs}, Training Loss:{running_loss:0.1f}, Time:{time()-start:0.1f}s")

print('Finished Training')

Epoch:1/50, Training Loss:185.2, Time:77.5s
Epoch:2/50, Training Loss:122.7, Time:76.6s
Epoch:3/50, Training Loss:91.5, Time:76.0s
Epoch:4/50, Training Loss:65.9, Time:77.4s
Epoch:5/50, Training Loss:48.5, Time:77.2s
Epoch:6/50, Training Loss:40.7, Time:77.3s
Epoch:7/50, Training Loss:29.3, Time:77.3s
Epoch:8/50, Training Loss:29.2, Time:77.4s
Epoch:9/50, Training Loss:33.0, Time:77.9s
Epoch:10/50, Training Loss:30.0, Time:76.4s
Epoch:11/50, Training Loss:18.3, Time:76.8s
Epoch:12/50, Training Loss:21.7, Time:77.1s
Epoch:13/50, Training Loss:19.1, Time:76.7s
Epoch:14/50, Training Loss:14.3, Time:76.7s
Epoch:15/50, Training Loss:13.8, Time:76.8s
Epoch:16/50, Training Loss:11.8, Time:76.6s
Epoch:17/50, Training Loss:18.4, Time:76.7s
Epoch:18/50, Training Loss:12.5, Time:77.2s
Epoch:19/50, Training Loss:8.1, Time:77.0s
Epoch:20/50, Training Loss:6.5, Time:77.7s
Epoch:21/50, Training Loss:11.1, Time:78.0s
Epoch:22/50, Training Loss:17.8, Time:77.1s
Epoch:23/50, Training Loss:12.8, Time:77.

In [18]:
# Checking accuracy on validation set

correct = 0
total = 0

# Variables to keep track of accuracy for each class
class_correct = [0 for _ in classes]
class_total = [0 for _ in classes]

net.eval()

with torch.no_grad():

    for data in val_loader:

        # Get the inputs and label data
        inp_data, label = data
        
        # Break up the inputs
        img, cmd = inp_data
        
        # Put data into the GPU
        img = img.to(device)
        cmd = cmd.to(device)
        label = label.to(device)


        # Predict
        output = net((img, cmd))
        
        # Assuming we always get batches
        for i in range(output.size()[0]):
                
            # Get the predicted most probable move
            move = torch.argmax(output[i])
                
            if move == label[i]:
                class_correct[label[i]] += 1
                class_total[label[i]] += 1
                correct +=1
            else:
                class_total[label[i]] += 1
            total += 1
        
# Calculate and output total set accuracy 
accuracy = correct / total
print(f"Accuracy on validation set: {correct}/{total} = {accuracy*100:.2f}%")

# Calculate and show accuracy for each class
for i, cls in enumerate(classes):
    ccorrect = class_correct[i]
    ctotal = class_total[i]
    caccuracy = ccorrect / ctotal
    print(f"  Accuracy on {cls:>5} class: {ccorrect}/{ctotal} = {caccuracy*100:.2f}%")

Accuracy on validation set: 1297/1399 = 92.71%
  Accuracy on  left class: 192/227 = 84.58%
  Accuracy on right class: 201/243 = 82.72%
  Accuracy on straight class: 904/929 = 97.31%


In [19]:
# Save trained model given PATH
PATH = 'cmd_torch_next50_.pth'
torch.save(net.state_dict(), PATH)